In [ ]:
!pip install LTNtorch

In [ ]:
!pip install --break-system-packages torch torchvision
!pip install --break-system-packages ltn-torch

ERROR: Could not find a version that satisfies the requirement ltn-torch (from versions: none)
ERROR: No matching distribution found for ltn-torch


In [ ]:
import os, random, requests
from io import BytesIO
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms

import ltn  # da LTNtorch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)


Device: cpu


In [ ]:
import os, requests
from PIL import Image
from io import BytesIO

urls_dogs = [
    "https://jpimg.com.br/uploads/2024/11/8-racas-de-cachorros-mais-carinhosas-do-mundo.jpg",
    "https://static.nationalgeographicbrasil.com/files/styles/image_3200/public/01-domesticated-dog.webp?w=760&h=507",
    "https://adimax.com.br/wp-content/uploads/2024/08/cachorro-de-pelo-curto-confira-dicas-e-cuidados.jpg",
    "https://www.otempo.com.br/content/dam/otempo/editorias/pets/2024/2/pets-pets-animais-de-estimacao-cachorro-edicase-1713429167.jpeg",
    "https://marketplace.canva.com/MACxToiJtUQ/1/thumbnail_large-1/canva-jack-russell-terrier-MACxToiJtUQ.png",
] * 4

urls_cats = [
    "https://www.petz.com.br/blog/wp-content/uploads/2019/07/vida-de-gato.jpg",
    "https://plus.unsplash.com/premium_photo-1667030474693-6d0632f97029?fm=jpg&q=60&w=3000&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8Z2F0b3xlbnwwfHwwfHx8MA%3D%3D",
    "https://www.petsupport.com.br/wp-content/uploads/2022/02/pelo-do-gato-1024x640.jpg",
    "https://media.istockphoto.com/id/1029265614/pt/foto/black-cat-lying-on-the-grass-in-the-garden-against-purple-flowers.jpg?s=612x612&w=0&k=20&c=rn_UzAWBGC8t3s0Yg_hz08qEWb7ghLIVOWJLRshKGPA=",
    "https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.doglife.com.br%2Fblog%2Fgato-filhote-tudo-que-voce-precisa-saber-para-cuidar-bem-61aa5b4f5448461cf9e0a54b%2F&psig=AOvVaw3HWamIbi37CrYZIcmCueNc&ust=1763864836097000&source=images&cd=vfe&opi=89978449&ved=0CBUQjRxqFwoTCJj70tnahJEDFQAAAAAdAAAAABAt",
] * 4


def baixar_dataset():
    for split in ["train", "val"]:
        for cls in ["dogs", "cats"]:
            os.makedirs(f"data/{split}/{cls}", exist_ok=True)

    def salvar_imgs(urls, cls):
        for i, url in enumerate(urls):
            try:
                img = Image.open(BytesIO(requests.get(url).content))
                split = "train" if i < 15 else "val"  # 15 treino, 5 validação
                img.save(f"data/{split}/{cls}/{cls}_{i}.jpg")
            except:
                pass

    salvar_imgs(urls_dogs, "dogs")
    salvar_imgs(urls_cats, "cats")

baixar_dataset()
print("Dataset criado com sucesso!")


Dataset criado com sucesso!


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import ltn

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cpu')

In [ ]:
IMG_SIZE = 128
BATCH_SIZE = 16

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder("data/train", transform=train_transform)
val_ds   = datasets.ImageFolder("data/val",   transform=val_transform)

print("Classes e índices:", train_ds.class_to_idx)
# Esperado: {'cats': 0, 'dogs': 1}

idx_cat = train_ds.class_to_idx["cats"]
idx_dog = train_ds.class_to_idx["dogs"]

def subset_by_class(dataset, class_idx):
    # Usa .targets (PyTorch >=1.7) ou .samples
    if hasattr(dataset, "targets"):
        indices = [i for i, y in enumerate(dataset.targets) if y == class_idx]
    else:
        indices = [i for i, (_, y) in enumerate(dataset.samples) if y == class_idx]
    return Subset(dataset, indices)

train_dogs_ds = subset_by_class(train_ds, idx_dog)
train_cats_ds = subset_by_class(train_ds, idx_cat)
val_dogs_ds   = subset_by_class(val_ds, idx_dog)
val_cats_ds   = subset_by_class(val_ds, idx_cat)

print("Tamanhos treino:", len(train_dogs_ds), "dogs |", len(train_cats_ds), "cats")
print("Tamanhos val:   ", len(val_dogs_ds),   "dogs |", len(val_cats_ds),   "cats")

train_dogs_loader = DataLoader(train_dogs_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
train_cats_loader = DataLoader(train_cats_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

val_dogs_loader   = DataLoader(val_dogs_ds,   batch_size=BATCH_SIZE, shuffle=False)
val_cats_loader   = DataLoader(val_cats_ds,   batch_size=BATCH_SIZE, shuffle=False)


Classes e índices: {'cats': 0, 'dogs': 1}
Tamanhos treino: 12 dogs | 12 cats
Tamanhos val:    4 dogs | 4 cats


In [ ]:
class DogCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 64x64
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # 32x32
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),# 16x16
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2) # 8x8
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, 1)   # um logit
        )

    def forward(self, x):
        return self.classifier(self.features(x))

model = DogCNN().to(DEVICE)
print(model)


DogCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [ ]:
Dog = ltn.Predicate(model)
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Forall = ltn.Quantifier(
    ltn.fuzzy_ops.AggregPMeanError(p=2),
    quantifier="f"
)
SatAgg = ltn.fuzzy_ops.SatAgg()

optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
import numpy as np

EPOCHS = 5

for epoch in range(EPOCHS):
    model.train()
    losses = []

    for (dogs, _), (cats, _) in zip(train_dogs_loader, train_cats_loader):
        dogs, cats = dogs.to(DEVICE), cats.to(DEVICE)

        d = ltn.Variable("d", dogs)
        c = ltn.Variable("c", cats)

        sat = SatAgg(
            Forall(d, Dog(d)),
            Forall(c, Not(Dog(c)))
        )

        loss = 1 - sat

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    print(f"Epoch {epoch+1}/{EPOCHS} — Loss: {np.mean(losses):.4f}")


Epoch 1/5 — Loss: nan
Epoch 2/5 — Loss: nan
Epoch 3/5 — Loss: nan
Epoch 4/5 — Loss: nan
Epoch 5/5 — Loss: nan


/usr/local/lib/python3.12/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.12/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import torch.nn.functional as F

def acc(loader, label):
    correct, total = 0, 0
    for imgs, _ in loader:
        imgs = imgs.to(DEVICE)
        logits = model(imgs)
        preds = (torch.sigmoid(logits).view(-1) >= 0.5).long()
        correct += (preds == label).sum().item()
        total += len(preds)
    return correct / total if total else 0

print("Acurácia Dogs:", acc(val_dogs_loader, torch.tensor(1)))
print("Acurácia Cats:", acc(val_cats_loader, torch.tensor(0)))


Acurácia Dogs: 1.0
Acurácia Cats: 0.0
